# Preparing the enviroment and mounting data

In [1]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
#from google.colab import drive
#drive.mount("/content/drive/")
#cd drive/My Drive/SFU/Project4-Spring2021
!ls

bartpy			 known_snps.txt  train.py	       x_colnames.npy
eval.py			 __pycache__	 x_clinical_names.npy  x_snps.npy
known_snps_fullname.txt  sample_data	 x_clinical.npy        y.npy


In [2]:
!git clone https://github.com/JakeColtman/bartpy.git
#can i just clone parkca? check the differences between my train and the one on the package

fatal: destination path 'bartpy' already exists and is not an empty directory.


In [3]:
# reading snps file
#libraries
import pandas as pd
import numpy as np 
from scipy import sparse
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import sys
sys.path.insert(0,'bartpy/')
from bartpy.sklearnmodel import SklearnModel

data_preprocessing = False
run_learners = True

# BCCH Data Analysis 

## Data 
There are two options: 
1. Pre-processing: require original files on the server and perform the pre-processing / filtering 
2. Load files saved on local machine 


In [4]:
path = '/content/'
if data_preprocessing:  
  print('Option 1: Starting pre-processing:')
  import data_preprossing_functions as dpf
  y, x_clinical, x_snps, x_colnames, x_clinical_names = dpf.run_preprocessing(path, True)
  x_clinical = x_clinical.astype('float64')
  print('Done!')
  print('Shapes:',len(y), sum(y), x_clinical.shape, x_snps.shape, len(x_colnames), len(x_clinical_names))
  #318 (318, 42) (318, 16641) 16641
else: 
  print('Option 2: Reading files')
  y =  np.load('y.npy')
  x_clinical = np.load('x_clinical.npy',allow_pickle =True)
  x_clinical = x_clinical.astype('float64')
  x_clinical_names = np.load('x_clinical_names.npy',allow_pickle =True)
  x_snps = np.load('x_snps.npy',allow_pickle =True)
  x_colnames  = np.load('x_colnames.npy',allow_pickle =True)
  print('Done!')
  print('Shapes:',len(y), sum(y), x_clinical.shape, x_snps.shape, len(x_colnames), len(x_clinical_names))
 

Option 2: Reading files
Done!
Shapes: 302 187 (302, 41) (302, 16195) 16195 41


## ParKCa

1. Learners: Deconfounder, BART



In [5]:
import train as parkca
import eval as evaluation

#if run_learners:
#  level1data = parkca.learners_bcch(path_output = path, learners = ['DA'], X = x_snps, y = y, 
#                                    colnamesX = x_colnames,causes = 'snps', Z = x_clinical, colnamesZ = x_clinical_names)

X = np.concatenate([x_clinical,x_snps], axis = 1)
print(X.shape)

X = preprocessing.StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


Instructions for updating:
non-resource variables are not supported in the long term
(302, 16236)


In [ ]:
#model_da = parkca.learner_deconfounder_algorithm(X_train, X_test, y_train, y_test, 10)
#model_bart = parkca.learner_BART(X_train, X_test, y_train, y_test)
#coef = parkca.learners(['DA','BART','noise'],X,y,x_colnames) #bart is time consuming, bad for testing
level1data = parkca.learners(['DA','noise'],X,y,x_colnames)




Learner: DA
... There are  1  versions of DA
...... Version 1/ 1
Running DA
... Done Holdout


In [ ]:
#
#bart.fit()
#coef, coef_c, roc  = model_da.fit()

In [ ]:
level1data.shape

In [ ]:
level1data.head()

In [ ]:
ks = pd.read_csv(path+'known_snps_fullname.txt', header = None)[0].values
level1data['y'] = 0
level1data['y'] = [1 if i in ks else 0 for i in level1data['causes'].values]

In [ ]:
qav, q_ = evaluation.diversity(['DA_15','noise'], level1data, 'y')
print('Diversity (Best when <= 0): ', qav)

In [ ]:
level1data.head()

In [ ]:
level1data.set_index('causes', inplace = True, drop = True) 
level1data = parkca.data_norm(level1data)


In [ ]:
roc, output = parkca.meta_learner(level1data, ['rf','lr','nn','upu'],'y')
